# Tokenizer

In [9]:
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, decoders
from tokenizers.processors import TemplateProcessing
import json

# === Load data ===
with open("baseline-dataset.jsonl", "r", encoding="utf-8") as f:
    texts = []
    for line in f:
        data = json.loads(line)
        texts.append(data["prompt"])
        texts.append(data["response"])

# === Write to plain text (required by tokenizer trainer) ===
with open("tokenizer_corpus.txt", "w", encoding="utf-8") as f:
    for text in texts:
        f.write(text.strip() + "\n")

# === Init tokenizer ===
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
trainer = trainers.BpeTrainer(
    vocab_size=8000,
    show_progress=True,
    special_tokens=["<pad>", "<s>", "</s>", "<unk>"],
)

# === Train ===
tokenizer.train(["tokenizer_corpus.txt"], trainer)

# === Post-processing untuk auto menambahkan <s> dan </s> saat encoding
tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    pair="<s> $A </s> <s> $B </s>",
    special_tokens=[
        ("<s>", tokenizer.token_to_id("<s>")),
        ("</s>", tokenizer.token_to_id("</s>")),
    ],
)

tokenizer.decoder = decoders.ByteLevel()

# === Save ===
tokenizer.save("tokenizer-agrolens.json")   
print("✅ Tokenizer saved to tokenizer-agrolens.json")




✅ Tokenizer saved to tokenizer-agrolens.json


In [10]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("tokenizer-agrolens.json")
enc = tokenizer.encode("Apa itu penyakit blast?")
print(enc.tokens)
print(enc.ids)

['<s>', 'ĠApa', 'Ġitu', 'Ġpenyakit', 'Ġblast', '?', '</s>']
[1, 187, 353, 141, 212, 10, 2]


# GPTModel

In [20]:
import tensorflow as tf
from tensorflow.keras import layers


class AgroLensGPT(tf.keras.Model):
    def __init__(
        self,
        vocab_size,
        max_length=512,
        d_model=256,
        n_heads=4,
        n_layers=4,
        dropout=0.1,
    ):
        super(AgroLensGPT, self).__init__()
        self.token_embed = layers.Embedding(input_dim=vocab_size, output_dim=d_model)
        self.pos_embed = layers.Embedding(input_dim=max_length, output_dim=d_model)

        self.decoder_blocks = [
            tf.keras.layers.LayerNormalization(epsilon=1e-6),
        ]
        self.decoder_layers = []
        for _ in range(n_layers):
            self.decoder_layers.append(
                [
                    layers.MultiHeadAttention(num_heads=n_heads, key_dim=d_model),
                    layers.Dropout(dropout),
                    layers.LayerNormalization(epsilon=1e-6),
                    layers.Dense(d_model * 4, activation="relu"),
                    layers.Dense(d_model),
                    layers.Dropout(dropout),
                    layers.LayerNormalization(epsilon=1e-6),
                ]
            )

        self.final_ln = layers.LayerNormalization(epsilon=1e-6)
        self.output_head = layers.Dense(vocab_size)

    def call(self, x, training=False):
        B, T = tf.shape(x)[0], tf.shape(x)[1]
        token_emb = self.token_embed(x)  # (B, T, d_model)
        positions = tf.range(start=0, limit=T, delta=1)
        pos_emb = self.pos_embed(positions)  # (T, d_model)
        pos_emb = tf.expand_dims(pos_emb, axis=0)  # (1, T, d_model)
        h = token_emb + pos_emb  # (B, T, d_model)

        # Causal mask
        causal_mask = tf.linalg.band_part(tf.ones((T, T)), -1, 0)  # (T, T)
        causal_mask = tf.cast(causal_mask, dtype=tf.bool)

        for mha, drop1, ln1, ff1, ff2, drop2, ln2 in self.decoder_layers:
            attn_output = mha(h, h, h, attention_mask=causal_mask, use_causal_mask=True)
            attn_output = drop1(attn_output, training=training)
            h = ln1(h + attn_output)

            ffn_output = ff1(h)
            ffn_output = ff2(ffn_output)
            ffn_output = drop2(ffn_output, training=training)
            h = ln2(h + ffn_output)

        h = self.final_ln(h)
        logits = self.output_head(h)  # (B, T, vocab_size)
        return logits

2025-06-04 20:04:17.000752: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 20:04:17.152193: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749038657.205823    1871 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749038657.222923    1871 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749038657.359340    1871 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [21]:
model = AgroLensGPT(vocab_size=8000)
display(model.summary())
sample_input = tf.random.uniform((2, 64), minval=0, maxval=8000, dtype=tf.int32)
logits = model(sample_input)
print(logits.shape)  # Expected: (2, 64, 8000)

I0000 00:00:1749038666.870329    1871 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5255 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "agro_lens_gpt"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multi_head_attention            │ ?                      │   0 (unbuilt) │
│ (MultiHeadAttention)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multi_head_attention_1          │ ?                      │   0 (unbuilt) │
│ (MultiHeadAttention)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_4           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ multi_head_attention_2          │ ?                      │   0 (unbuilt) │
│ (MultiHeadAttention)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_5           │ ?                      │   0 (unbuilt) │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None

/home/ruvne/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'agro_lens_gpt', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


(2, 64, 8000)


# Loader dan Tokenizer

In [23]:
import json
import tensorflow as tf
from tokenizers import Tokenizer


class AgroDatasetTF(tf.data.Dataset):
    def __new__(cls, path, tokenizer_path, max_len=256):
        tokenizer = Tokenizer.from_file(tokenizer_path)
        samples = []

        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                data = json.loads(line)
                prompt = data["prompt"]
                response = data["response"]
                combined = f"{prompt} {response}"

                # Tokenize and truncate
                ids = tokenizer.encode(combined).ids[:max_len]

                if len(ids) >= 2:  # minimal length to create input/label
                    input_ids = ids[:-1]
                    labels = ids[1:]
                    samples.append((input_ids, labels))

        # Convert to TensorFlow tensors
        def gen():
            for input_ids, labels in samples:
                yield {
                    "input_ids": tf.constant(input_ids, dtype=tf.int32),
                    "labels": tf.constant(labels, dtype=tf.int32),
                }

        return tf.data.Dataset.from_generator(
            gen,
            output_signature={
                "input_ids": tf.TensorSpec(shape=(None,), dtype=tf.int32),
                "labels": tf.TensorSpec(shape=(None,), dtype=tf.int32),
            },
        )

In [35]:
dataset = AgroDatasetTF(
    "baseline-dataset.jsonl", "tokenizer-agrolens.json", max_len=256
)
dataset = dataset.padded_batch(8, padded_shapes={"input_ids": [None], "labels": [None]})
for batch in dataset.take(1):
    print(batch["input_ids"].shape, batch["labels"].shape)

(8, 50) (8, 50)


In [27]:
import tensorflow as tf

# --- Hyperparameters ---
BATCH_SIZE = 16
EPOCHS = 100
LR = 3e-4
MAX_LEN = 256

# --- Dataset ---
dataset = AgroDatasetTF(
    "baseline-dataset.jsonl", "tokenizer-agrolens.json", max_len=MAX_LEN
)
dataset = dataset.padded_batch(
    BATCH_SIZE, padded_shapes={"input_ids": [None], "labels": [None]}
)
dataset = dataset.shuffle(1000).prefetch(tf.data.AUTOTUNE)

# --- Model ---
model = AgroLensGPT(vocab_size=8000)
optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction="none"
)


# --- Custom Training Loop ---
@tf.function
def train_step(input_ids, labels):
    with tf.GradientTape() as tape:
        logits = model(input_ids, training=True)
        # Shifted label loss, ignoring padding (-100 equivalent in PyTorch)
        mask = tf.cast(labels != -100, tf.float32)
        loss_values = loss_fn(labels, logits)
        loss = tf.reduce_sum(loss_values * mask) / tf.reduce_sum(mask)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


# --- Training Loop ---
for epoch in range(EPOCHS):
    total_loss = 0.0
    steps = 0

    for batch in dataset:
        input_ids = batch["input_ids"]
        labels = batch["labels"]
        loss = train_step(input_ids, labels)
        total_loss += loss.numpy()
        steps += 1

    avg_loss = total_loss / steps
    print(f"📘 Epoch {epoch+1}: Loss = {avg_loss:.4f}")

# --- Save weights ---
model.save_weights("agrolens_model_tf.weights.h5")

/home/ruvne/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'agro_lens_gpt_2', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


📘 Epoch 1: Loss = 9.0075
📘 Epoch 2: Loss = 8.5637
📘 Epoch 3: Loss = 8.2816
📘 Epoch 4: Loss = 8.0811
📘 Epoch 5: Loss = 7.9130
📘 Epoch 6: Loss = 7.7529
📘 Epoch 7: Loss = 7.5804
📘 Epoch 8: Loss = 7.4032
📘 Epoch 9: Loss = 7.2479
📘 Epoch 10: Loss = 7.0940
📘 Epoch 11: Loss = 6.9383
📘 Epoch 12: Loss = 6.7915
📘 Epoch 13: Loss = 6.6436
📘 Epoch 14: Loss = 6.4987
📘 Epoch 15: Loss = 6.3572
📘 Epoch 16: Loss = 6.2178
📘 Epoch 17: Loss = 6.0773
📘 Epoch 18: Loss = 5.9417
📘 Epoch 19: Loss = 5.7987
📘 Epoch 20: Loss = 5.6653
📘 Epoch 21: Loss = 5.5315
📘 Epoch 22: Loss = 5.3950
📘 Epoch 23: Loss = 5.2705
📘 Epoch 24: Loss = 5.1387
📘 Epoch 25: Loss = 5.0098
📘 Epoch 26: Loss = 4.8827
📘 Epoch 27: Loss = 4.7597
📘 Epoch 28: Loss = 4.6333


2025-06-04 20:06:44.844160: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


📘 Epoch 29: Loss = 4.5106
📘 Epoch 30: Loss = 4.3879
📘 Epoch 31: Loss = 4.2705
📘 Epoch 32: Loss = 4.1596
📘 Epoch 33: Loss = 4.0449
📘 Epoch 34: Loss = 3.9318
📘 Epoch 35: Loss = 3.8242
📘 Epoch 36: Loss = 3.7182
📘 Epoch 37: Loss = 3.6160
📘 Epoch 38: Loss = 3.5097
📘 Epoch 39: Loss = 3.4055
📘 Epoch 40: Loss = 3.3103
📘 Epoch 41: Loss = 3.2166
📘 Epoch 42: Loss = 3.1141
📘 Epoch 43: Loss = 3.0230
📘 Epoch 44: Loss = 2.9328
📘 Epoch 45: Loss = 2.8456
📘 Epoch 46: Loss = 2.7583
📘 Epoch 47: Loss = 2.6735
📘 Epoch 48: Loss = 2.5864
📘 Epoch 49: Loss = 2.5027
📘 Epoch 50: Loss = 2.4270
📘 Epoch 51: Loss = 2.3455
📘 Epoch 52: Loss = 2.2751
📘 Epoch 53: Loss = 2.1979
📘 Epoch 54: Loss = 2.1196
📘 Epoch 55: Loss = 2.0493
📘 Epoch 56: Loss = 1.9794
📘 Epoch 57: Loss = 1.9102
📘 Epoch 58: Loss = 1.8466
📘 Epoch 59: Loss = 1.7841
📘 Epoch 60: Loss = 1.7205
📘 Epoch 61: Loss = 1.6596
📘 Epoch 62: Loss = 1.5985
📘 Epoch 63: Loss = 1.5412
📘 Epoch 64: Loss = 1.4877
📘 Epoch 65: Loss = 1.4306
📘 Epoch 66: Loss = 1.3782
📘 Epoch 67: 

In [32]:
import tensorflow as tf
from tokenizers import Tokenizer

# --- Konstanta dan Load Model ---
MODEL_PATH = "agrolens_model_tf.weights.h5"
TOKENIZER_PATH = "tokenizer-agrolens.json"
VOCAB_SIZE = 8000
MAX_LEN = 128

model = AgroLensGPT(vocab_size=VOCAB_SIZE)
dummy_input = tf.constant([[1] * 64], dtype=tf.int32)  # bentuk (1, 64)
_ = model(dummy_input)  # memanggil forward p
model.load_weights(MODEL_PATH)
tokenizer = Tokenizer.from_file(TOKENIZER_PATH)

/home/ruvne/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'agro_lens_gpt_7', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [33]:
def generate(prompt: str, max_new_tokens=50):
    # Tokenisasi prompt
    input_ids = tokenizer.encode(prompt).ids
    input_tensor = tf.constant([input_ids], dtype=tf.int32)

    for _ in range(max_new_tokens):
        # Loloskan input ke model
        logits = model(input_tensor, training=False)

        # Ambil token berikutnya dari distribusi logit terakhir
        next_token = tf.argmax(logits[:, -1, :], axis=-1, output_type=tf.int32)

        # Tambahkan ke input_tensor
        input_tensor = tf.concat(
            [input_tensor, tf.expand_dims(next_token, axis=1)], axis=1
        )

        # Jika token akhir ditemukan
        if tokenizer.token_to_id("</s>") in next_token.numpy():
            break

        # Batasi panjang maksimum
        if input_tensor.shape[1] >= MAX_LEN:
            break

    # Decode output
    output_ids = input_tensor[0].numpy().tolist()
    return tokenizer.decode(output_ids)

In [34]:
import ipywidgets as widgets
from IPython.display import display, Markdown

input_box = widgets.Text(
    value="Apa itu penyakit blast?",
    placeholder="Tulis pertanyaan di sini...",
    description="❓ Pertanyaan:",
    layout=widgets.Layout(width="100%"),
)

output_box = widgets.Output()
generate_button = widgets.Button(
    description="Jawab 🚀", button_style="success", layout=widgets.Layout(width="15%")
)


def on_generate_clicked(b):
    prompt = input_box.value
    response = generate(prompt)
    output_box.clear_output()
    with output_box:
        display(
            Markdown(
                f"### 🧑 Kamu: \n{prompt}\n---\n### 🌾 AgroLens Menjawab:\n{response}"
            )
        )


generate_button.on_click(on_generate_clicked)

# Tampilkan
display(widgets.VBox([input_box, generate_button, output_box]))